In [1]:
import pandas as pd
import datetime

Based on the attached data, answer the following two questions. Explain your answer. 
1. Customers who made their first order in which calendar week, generated the most revenue in their 3rd week? Your answer should be a week number. Week count starts with 1. 
2. In which month, did we get the customers with highest client retention in their 3rd calendar month? Your answer should be a month number. Month count starts with 1.

In [2]:
df=pd.read_csv('test-data.csv')

In [3]:
df.head()

,order_id,created,user_id,payment
0,3178039,2018-03-09 19:12:13,1066,1090
1,2940729,2018-02-02 13:56:01,4821,349
2,4260454,2018-08-14 13:32:25,4831,673
3,3468644,2018-04-24 14:24:58,9563,893
4,2842424,2018-01-15 15:44:33,12189,480


Сколько записей?

In [4]:
df.shape

(59998, 4)

Сколько пользователей?

In [5]:
df.user_id.nunique()

13377

# Задание 1

In [6]:
#преобразовываем строку в дату из datetime.Теперь можно использовать методы Datetime для работы с датами
df['order_day']=pd.to_datetime(df['created'])

In [7]:
df['order_week']=df['order_day'].dt.week

In [8]:
#оставляем только дату заказа (убираем время)
df['order_day']=df['order_day'].dt.date

In [9]:
#создаем таблицу  c данными по пользователю
df_users=pd.DataFrame(df.groupby('user_id').agg('order_week').min()).rename(columns={'order_week':'first_order_week'}) \
                    .reset_index()
df_users.head()

,user_id,first_order_week
0,1066,10
1,4821,5
2,4831,33
3,9563,17
4,12189,3


In [10]:
#добавляем в основную таблицу данные по тому, в какой номер недели была совершена первая покупка
df=df.merge(df_users, left_on='user_id', right_on='user_id', how='outer')

In [11]:
#добавляем в основную таблицу данные по тому, в какую неделю после первой была покупка
df['week_number_after_first_week']=df['order_week']-df['first_order_week']

In [12]:
#оставляем только покупки на 3 неделе после первой покупки и сохраняем в отдельный df
res_df=df.query('week_number_after_first_week==3').head(100)

In [13]:
res_df.head()

,order_id,created,user_id,payment,order_day,order_week,first_order_week,week_number_after_first_week
86,4107374,2018-07-25 17:16:39,73963,360,2018-07-25,30,27,3
124,4125549,2018-07-27 14:09:45,98611,640,2018-07-27,30,27,3
152,3825331,2018-06-18 9:16:59,113961,199,2018-06-18,25,22,3
153,3836112,2018-06-19 11:25:25,113961,552,2018-06-19,25,22,3
154,3857838,2018-06-21 14:51:40,113961,450,2018-06-21,25,22,3


In [14]:
res_payment=pd.DataFrame(res_df.groupby('first_order_week').agg('payment').sum())

In [15]:
res_users=pd.DataFrame(res_df.groupby('first_order_week').agg('user_id').count())

In [16]:
result=res_users.merge(res_payment, left_on='first_order_week', right_on='first_order_week', how='outer')
result=result.rename(columns={'user_id':'users','payment':'payment_sum'})

In [17]:
result['payment_per_user']=round(result['payment_sum']/result['users'],2)

In [35]:
result.reset_index().sort_values('payment_sum', ascending=False)

,first_order_week,users,payment_sum,payment_per_user
8,22,11,7550,686.36
4,9,15,6633,442.20
3,7,9,5845,649.44
1,3,9,5716,635.11
9,23,11,4566,415.09
2,4,8,4181,522.62
17,35,8,3864,483.00
11,27,6,3429,571.50
18,36,4,2512,628.00
7,21,5,2041,408.20


# Answer: 22 week

_______________________________________________________

# Задание 2

In which month, did we get the customers with highest client retention in their 3rd calendar month? Your answer should be a month number. Month count starts with 1.

In [19]:
data=pd.read_csv('test-data.csv')

In [20]:
data['order_day']=pd.to_datetime(data['created'])

In [21]:
data.head()

,order_id,created,user_id,payment,order_day
0,3178039,2018-03-09 19:12:13,1066,1090,2018-03-09 19:12:13
1,2940729,2018-02-02 13:56:01,4821,349,2018-02-02 13:56:01
2,4260454,2018-08-14 13:32:25,4831,673,2018-08-14 13:32:25
3,3468644,2018-04-24 14:24:58,9563,893,2018-04-24 14:24:58
4,2842424,2018-01-15 15:44:33,12189,480,2018-01-15 15:44:33


In [22]:
data['order_month']=data['order_day'].dt.month

In [23]:
data.head()

,order_id,created,user_id,payment,order_day,order_month
0,3178039,2018-03-09 19:12:13,1066,1090,2018-03-09 19:12:13,3
1,2940729,2018-02-02 13:56:01,4821,349,2018-02-02 13:56:01,2
2,4260454,2018-08-14 13:32:25,4831,673,2018-08-14 13:32:25,8
3,3468644,2018-04-24 14:24:58,9563,893,2018-04-24 14:24:58,4
4,2842424,2018-01-15 15:44:33,12189,480,2018-01-15 15:44:33,1


In [24]:
data_users=pd.DataFrame(data.groupby('user_id').agg('order_month').min()).rename(columns={'order_month':'first_order_month'}) \
                    .reset_index()
data_users.head()

,user_id,first_order_month
0,1066,3
1,4821,2
2,4831,8
3,9563,4
4,12189,1


In [25]:
data=data.merge(data_users, left_on='user_id', right_on='user_id', how='outer')

In [26]:
data['month_number_after_first_month']=data['order_month']-data['first_order_month']

In [27]:
users_month=pd.DataFrame(data.groupby('first_order_month').agg('user_id').count()) \
                             .rename(columns={'user_id':'1_month_users_count'}) \
                             .reset_index()

In [28]:
users_month.head()

,first_order_month,1_month_users_count
0,1,7694
1,2,9082
2,3,8620
3,4,7677
4,5,6844


In [29]:
users_month_3=pd.DataFrame(data.query('month_number_after_first_month==3').groupby('first_order_month') \
                               .agg('user_id').count())  \
                               .rename(columns={'user_id':'3_month_users_count'}) \
                               .reset_index()

In [30]:
users_month=users_month.merge(users_month_3, left_on='first_order_month', right_on='first_order_month', how='outer' )

In [31]:
users_month.fillna(0)

,first_order_month,1_month_users_count,3_month_users_count
0,1,7694,834.0
1,2,9082,1028.0
2,3,8620,930.0
3,4,7677,1065.0
4,5,6844,994.0
5,6,5928,1034.0
6,7,5164,526.0
7,8,4764,0.0
8,9,3124,0.0
9,10,1101,0.0


In [32]:
users_month['Retention']=round(users_month['3_month_users_count']/users_month['1_month_users_count'],2)

In [36]:
users_month.fillna(0).sort_values('Retention', ascending=False)

,first_order_month,1_month_users_count,3_month_users_count,Retention
5,6,5928,1034.0,0.17
4,5,6844,994.0,0.15
3,4,7677,1065.0,0.14
0,1,7694,834.0,0.11
1,2,9082,1028.0,0.11
2,3,8620,930.0,0.11
6,7,5164,526.0,0.10
7,8,4764,0.0,0.00
8,9,3124,0.0,0.00
9,10,1101,0.0,0.00


# Answer: 6 month